In [0]:
# ler a tabela de previsão horária imperial
df_clima = spark.read.table("samples.accuweather.forecast_hourly_imperial")

# Visualizar os dados
display(df_clima.limit(10))

In [0]:
# Selecionar colunas estratégicas para monitoramento de ativos
df_monitoramento = df_clima.select(
    "datetime_valid_local", 
    "temperature", 
    "humidity_relative", 
    "cloud_cover_perc_total"
)

# Criando uma view
df_monitoramento.createOrReplaceTempView("v_monitoramento_ativos")

display(df_monitoramento.limit(5))

In [0]:
%sql
SELECT 
    datetime_valid_local,
    temperature,
    humidity_relative,
    cloud_cover_perc_total,
    CASE 
        WHEN temperature > 90 THEN 'CRÍTICO: Superaquecimento de Ativos'
        WHEN humidity_relative > 85 THEN 'ALERTA: Risco de Condensação/Curto'
        WHEN cloud_cover_perc_total > 95 THEN 'ATENÇÃO: Baixa Eficiência Solar (Backup)'
        ELSE 'Operação Normal'
    END as status_operacional
FROM v_monitoramento_ativos
ORDER BY datetime_valid_local DESC

Databricks visualization. Run in Databricks to view.

In [0]:

df_monitoramento.write.mode("overwrite").saveAsTable("default.monitoramento_final_gold")